<a href="https://colab.research.google.com/github/fukushun1994/RNN-Anime-model/blob/master/RNN_Anime_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境構築


## GPUの設定

ランタイム > ランタイムのタイプを変更 で ハードウェアアクセラレータをGPUに設定



# データの準備


## Google Drive のマウント

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## データの取得

In [9]:
import zipfile
import numpy as np
from PIL import Image

DATA_DIR = '/content/gdrive/My Drive/YuyushikiData/'
ZIP_FILE_NAMES = ['ep1.zip']
SKIP = 1
threshold = 100

total_length = 0
data_list = []
for zip_file_name in ZIP_FILE_NAMES:
  data = []
  with zipfile.ZipFile(DATA_DIR+zip_file_name, 'r') as zp:
    images = sorted(zp.namelist())
    image_count = len(images)
    total_length += image_count
    print('open {} ({} images)'.format(zip_file_name, image_count))
    for i in range(0, image_count, SKIP):
      image_name = images[i]
      print('\r{}/{}'.format(i, image_count), end='')
      with zp.open(image_name, 'r') as fp:
        with Image.open(fp) as img:
          img_vec = np.array(img)[:,:,:3]/255
          if len(data) == 0 or np.sum(np.abs(img_vec - data[-1])) > threshold:
            data.append(img_vec)
  print('\nadded {} images'.format(len(data)))
  data_list.append(data)

image_shape = data_list[0][0].shape
print('')
print(image_shape)


open ep1.zip (26235 images)
26234/26235
added 9318 images

(72, 128, 3)


In [11]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        3.3G        6.3G        884K        3.2G         11G
Swap:            0B          0B          0B


## ジェネレータの作成

In [0]:
def data_generator(data, seq_len, batch_size):
  available_indices = []
  flatten_data = []
  offset = 0
  for d in data:
    available_indices += list(range(offset, offset+len(d)-seq_len))
    flatten_data += d
    offset += len(d)
  while True:
    indices = np.random.choice(available_indices, batch_size, replace=False)
    yield (
      np.stack([flatten_data[i:i+seq_len] for i in indices]),
      np.stack([flatten_data[i+1:i+1+seq_len] for i in indices])
    )


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

# モデルの構築


In [0]:
# code from https://github.com/keras-team/keras/blob/master/examples/conv_lstm.py

from keras.models import Sequential
from keras.layers import Flatten, Reshape, Dense
from keras.layers.core import Activation
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import TimeDistributed
import numpy as np
import pylab as plt

FILTER_NUM = 32
KERNEL_SIZE = (3, 3)

def create_model(input_shape):
  flat_dim = np.prod(input_shape[1:])
  seq = Sequential()
  seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                     input_shape=input_shape,
                     padding='same', return_sequences=True))
  seq.add(BatchNormalization())
  
  seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                     padding='same', return_sequences=True))
  seq.add(BatchNormalization())
  '''
  seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                     padding='same', return_sequences=True))
  seq.add(BatchNormalization())

  seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                     padding='same', return_sequences=True))
  seq.add(BatchNormalization())
  seq.add(Conv3D(filters=3, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
  '''
  seq.add(ConvLSTM2D(filters=3, kernel_size=KERNEL_SIZE,
                     padding='same', return_sequences=True))
  seq.compile(loss='binary_crossentropy', optimizer='adadelta')
  return seq

# 学習


In [0]:
from keras.callbacks import ModelCheckpoint

SEQ_LENGTH = 5
BATCH_SIZE = 10

seq = create_model((SEQ_LENGTH,) + image_shape)
cb = ModelCheckpoint('model.h5', save_best_only=True)
steps_per_epoch = 1000
seq.fit_generator(
    data_generator(data_list, SEQ_LENGTH, BATCH_SIZE),
    steps_per_epoch = steps_per_epoch,
    epochs=10,
    callbacks = [cb])
seq.save_weights('/tmp/weights.h5', overwrite=True)

Epoch 1/10
1000/1000 [==============================] - 596s 596ms/step - loss: 0.5372
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1000/1000 [==============================] - 590s 590ms/step - loss: 0.5048
Epoch 3/10
1000/1000 [==============================] - 592s 592ms/step - loss: 0.5027
Epoch 4/10
1000/1000 [==============================] - 591s 591ms/step - loss: 0.5032
Epoch 5/10
1000/1000 [==============================] - 591s 591ms/step - loss: 0.5022
Epoch 6/10
1000/1000 [==============================] - 591s 591ms/step - loss: 0.5008
Epoch 7/10
1000/1000 [==============================] - 591s 591ms/step - loss: 0.5006
Epoch 8/10
 608/1000 [=================>............] - ETA: 3:51 - loss: 0.5008

# 生成

In [0]:
!mkdir -p preds
!rm preds/*

PREDICT_LEN = 30

predict_model = create_model((1,) + image_shape)
predict_model.load_weights('/tmp/weights.h5')

seed = np.array(data_list[0][4000:4005])
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)
for i in range(seed.shape[1]):
  predict_model.predict(seed[:,i:i+1])
  
predictions = [seed[:,-1:]]
for i in range(PREDICT_LEN):
  last_pred = predictions[-1]
  pred = predict_model.predict(last_pred)
  predictions.append(pred)

for i in range(len(predictions)):
  for j in range(BATCH_SIZE):
    img_array = (predictions[i][j][0] * 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    img.save('preds/pred_{}_{}.png'.format(j, i), 'PNG')

!zip -r preds.zip preds

from google.colab import files

files.download('preds.zip')